## SparkML_GBT

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader, SQLContext
from sqlalchemy import create_engine
import pandas as pd
spark= SparkSession.builder.getOrCreate()
spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [2]:
import urllib.request
url = 'https://github.com/LeonardoPalaciosPando1996/Database/blob/Master/hmp.parquet?raw=true'
filename = 'hmp.parquet'
urllib.request.urlretrieve(url, filename)

('hmp.parquet', <http.client.HTTPMessage at 0x223c39c7f10>)

In [3]:
df = spark.read.parquet('hmp.parquet')
df.createOrReplaceTempView('df')

In [4]:
df_two_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")

In [5]:
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [7]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features_norm", maxIter=10)

In [8]:

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,gbt])


In [9]:
model = pipeline.fit(df_train)

In [10]:
prediction = model.transform(df_train)

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.9091999630894159

In [12]:
prediction = model.transform(df_test)

In [13]:
binEval.evaluate(prediction) 

0.9075144508670521